### General ideas

Ideas:
- Group by weekday (all locations) and location
- Group by month (all locations) and location
- Compute total per year and location
- Correlate with vacations and holidays (free time travel or work)
- Impact Corona
- https://meteostat.net/de/station/10513
- https://plotly.com/python/styling-plotly-express/
- https://binderhub.readthedocs.io/en/latest/
- https://www.netcup.de/vserver/vps.php
- https://nationaler-radverkehrsplan.de/de/taxonomy/term/4927
- http://www.eco-public.com/ParcPublic/?id=677

### Aufbau:
- Allgemeines: 
    - Wochentag
    - Monat
    - Jahr
- Einfluss des Wetters (Temperatur und Niederschlag)
- Einfluss von Corona
- Einfluss von Infrastrukturverbesserungen

### Abkürzungen
- DTV = Durchschnittliche tägliche Verkehrsstärke = RadfahrerInnen pro Werktag

#### Datenverfügbarkeit ab
- Venloer Straße: 08.2014
- Deutzer Brücke: 04.2016
- Neumarkt: 04.2016
- Stadtwald: 06.2015
- Volksgarten: 06.2018
- Suedstadt: 01.2013
- Vorgebirgspark: 06.2015
- Poller Wiesen: 06.2015
- Hohenzollernbrücke: 05.2016
- Niederländer Ufer: 06.2015

### Imports and function definitions

In [4]:
import requests
from datetime import date, datetime, timedelta
import pandas as pd
import numpy as np
import dateparser
import cufflinks as cf
import plotly.offline
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
import holidays
import statsmodels.api as sm

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
pd.options.plotting.backend = "plotly"

#### Parameter definition

In [5]:
start = datetime(2013, 1, 1)
end = datetime(2019, 12, 31)
bike_data_from_file = True
# plot style
style = "plotly_white"
bike_data_location = ""
weather_data_location = ""

In [26]:
# These definitions are already part of the results, but let's put them here nevertheless.
# Deutzer Brücke is somehow in the middle, neither work nor leisure station.
# Volksgarten is removed because data is available starting from 06.2018 only.

# Data availability work stations = 05.2016
work_stations = ["Neumarkt", "Bonner Straße", "Hohenzollernbrücke", "Venloer Straße"]
work_stations_data_start = datetime(2016, 5, 1)
# Data availability leisure stations = 06.2015
leisure_stations = ["Poller Wiesen", "Niederländer Ufer", "Stadtwald", "Vorgebirgspark"]
leisure_stations_data_start = datetime(2015, 6, 1)

data_start = datetime(2017, 1, 1)

In [27]:
def get_holidays_and_brueckentage(start: datetime, end: datetime):
    german_holidays = holidays.Germany(prov="NW", years=[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])
    list_of_holidays = []
    for date, name in sorted(german_holidays.items()):
        list_of_holidays.append(date)
        if date.weekday() == 1:
            list_of_holidays.append(date - timedelta(days=1))
        elif date.weekday() == 3:
            list_of_holidays.append(date + timedelta(days=1))
    
    return [d for d in list_of_holidays if not (d > end.date() or d < start.date())]

In [28]:
einwohner_koeln = pd.DataFrame(
    index=[
        datetime(2013, 1, 1), 
        datetime(2014, 1, 1),
        datetime(2015, 1, 1),
        datetime(2016, 1, 1),
        datetime(2017, 1, 1),
        datetime(2018, 1, 1),
        datetime(2019, 1, 1),
        datetime(2020, 1, 1)], 
    data=[
        1024373, 
        1034175,
        1046680,
        1060582,
        1075935,
        1080394,
        1085664,
       1087863],
columns=["einwohner"])

In [29]:
def load_bike_data_from_website(start: datetime, end: datetime):
    locations_to_ids = {
    "Universitätsstraße": "100059340",
    "Venloer Straße": "100023270", 
    "Zülpicher Straße": "100057124",
    "Deutzer Brücke": "100044729",
    "Neumarkt": "100030418",
    "Stadtwald": "100019757",
    "Volksgarten": "100045094",
    "Bonner Straße": "100023269",
    "Vorgebirgspark": "100019755",
    "Poller Wiesen": "100021727",
    "Hohenzollernbrücke": "100029854",
    "Niederländer Ufer": "100019758"   
    }
    
    result = pd.DataFrame()
    
    for location, idPdc in locations_to_ids.items():
        print(location)
        request_data = {"idOrganisme": "677", "idPdc": idPdc, 
                        "debut": start.strftime("%d/%m/%Y"), "fin": end.strftime("%d/%m/%Y"), 
                        "interval": "4", "pratiques": ""}
        response = requests.post(url="http://www.eco-public.com/ParcPublic/CounterData", data=request_data)
        raw_data = response.json()
        raw_data_transpose = np.array(raw_data[:-1]).T
        index = [dateparser.parse(i) for i in raw_data_transpose[0]]
        data = [float(i) for i in raw_data_transpose[1]]
        df = pd.DataFrame(index=index, data=data, columns=[location])
        result = pd.concat([result, df], axis=1)
    return result

In [30]:
def load_bike_data_from_file(start: datetime, end: datetime):
    df = pd.read_csv(
        bike_data_location, # Please specify path in the parameter section above
        index_col=0, 
        parse_dates=True)
    return df.loc[start:end, :]

In [31]:
def load_bike_data(start: datetime, 
                   end: datetime, 
                   specific_days: str="all", 
                   from_file: bool=True, 
                   remove_holidays: bool=True):
    """
    specific_days = weekday, saturday, sunday, all
    """
    if from_file is True:
        raw = load_bike_data_from_file(start, end)
    else:
        raw = load_bike_data_from_website(start, end)
    if remove_holidays is True:
        raw = raw.drop(get_holidays_and_brueckentage(start, end))
    raw = raw.loc[:, [location for location in list(raw.columns) 
                                if location not in ["Universitätsstraße", "Zülpicher Straße", "Volksgarten"]]]
    if specific_days == "weekday":
        return raw[raw.index.weekday <= 4]
    elif specific_days == "saturday":
        return raw[raw.index.weekday == 5]
    elif specific_days == "sunday":
        return raw[raw.index.weekday == 6]
    elif specific_days == "all":
        return raw

In [32]:
def load_weather_data(start: datetime, 
                      end: datetime, 
                      specific_days: str="all",
                      remove_holidays: bool=True):
    """
    specific_days = weekday, saturday, sunday, all or hourly (!)
    """
    raw = pd.read_csv( 
        weather_data_location, # Please specify path in the parameter section above
        index_col="Date", 
        parse_dates={"Date": [0,1]})
    raw = raw.loc[start: end, :].iloc[:, [0,3,6]]
    raw.columns = ["tavg", "prcp", "wspd"]
    
    if specific_days == "hourly":
        return raw
    tavg_wpd = raw.loc[:, ["tavg", "wspd"]].groupby(raw.index.date).mean()
    prcp = raw.loc[:, ["prcp"]].groupby(raw.index.date).sum()
    raw = pd.concat([tavg_wpd, prcp], 
                    axis=1)
    raw.index = pd.DatetimeIndex(raw.index)
    if remove_holidays is True:
        raw = raw.drop(get_holidays_and_brueckentage(start, end))
    if specific_days == "weekday":
        return raw[raw.index.weekday <= 4]
    elif specific_days == "saturday":
        return raw[raw.index.weekday == 5]
    elif specific_days == "sunday":
        return raw[raw.index.weekday == 6]
    elif specific_days == "all":
        return raw

In [33]:
def load_data(start: datetime, 
              end: datetime, 
              specific_days: str,
              remove_holidays: bool=True):
    return pd.concat([load_bike_data(start, end, specific_days, remove_holidays), 
                      load_weather_data(start, end, specific_days, remove_holidays)], 
                     axis=1)

### Einfluss der Wochentages
- Von Montag nach Dienstag/Mittwoch steigen die Werte leicht an
- Dienstag/Mittwoch zeigen bei den meisten Zählstationen den höchsten Wert
- Von Dienstag/Mittwoch sinkt der Wert
- Die Werte am Wochende sind immer niedriger als unter der Woche. Ausnahmen: Poller Wiese und Niederländer Ufer. Dort steigen die Werte Sonntags signifikant an (Freizeitverkehr) und sind sogar die höchsten der Woche.
- Starker Anstieg am Sonntag zeigen: Poller Wiese, Niederländer Ufer, Vorgebirgspark, Stadtwald
- Quotient aus DIV Mittwoch und Sonntag:
    - Je höher der Wert ist, desto mehr Personen fahren unter der Woche. Oder anders ausgedrückt: Je höher der Wert, desto eher handelt es sich um einen Zählpunkt, der vor allen Dingen den Berufsverkehr erfasst. Höchster Wert: Neumarkt.
    - Je niedriger der Wert, desto eher handelt es sich um eine Freizeitstrecke. Geringster Wert: Poller Wiesen.

In [34]:
dayOfWeek={0:'Montag', 1:'Dienstag', 2:'Mittwoch', 3:'Donnerstag', 4:'Freitag', 5:'Samstag', 6:'Sonntag'}
bike = load_bike_data(start, end, specific_days="all", remove_holidays=True)
bike_grouped_by_weekday = bike.groupby(bike.index.weekday).mean()
bike_grouped_by_weekday.index = bike_grouped_by_weekday.index.map(dayOfWeek)
bike_grouped_by_weekday.round(0).iplot()

In [35]:
# Finale Grafik!
df = bike_grouped_by_weekday.round(0).loc[:, ["Venloer Straße", "Neumarkt", "Poller Wiesen", "Niederländer Ufer"]]
fig = px.line(df, 
              template=style,
              labels={"value": "DIV (RadfahrerInnen pro Tag)",  "index": "Wochentag", "variable": "Zählstation"})
# Update title
fig.update_layout(
    title={'text': "Anzahl RadfahrerInnen pro Wochentag und Zählstation",'y':0.99,'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
# Update legend
fig.update_layout(
    font_family="Rockwell",
    legend=dict(title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center")
)
# Update y-axis
fig.update_yaxes(
        range = [500, 6000],
        dtick = 1000
)

fig.show()

In [36]:
# Finale Grafik!
df = (bike_grouped_by_weekday.loc["Mittwoch"] / bike_grouped_by_weekday.loc["Sonntag"]).sort_values()
fig = px.bar(df, 
              template=style,
              labels={"value": "DIV - Mittwoch / Sonntag",  "index": "Zählstation"})
# Update title
fig.update_layout(
    showlegend=False,
    title={'text': "Wie viele RadfahrerInnen fahren mittwochs im Vergleich zu sonntags?",'y':0.99,'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

## Einfluss des Monats
- Im Sommer (Apr - Sept) wird mehr Rad gefahren als im Winter (Okt - Mrz) (Überraschung :-))
- Wie viel weniger im Winter gefahren wird, unterscheidet sich stark von Zählstation zu Zählstation.
- In der Analyse der Wochentage haben wir Zählstationen unterteilt in Berufs- bzw. Freizeitverkehr: Zählstationen mit viel Berufsverkehr (Neumarkt, Venloer Straße, Hohenzollernbrücke, Bonner Straße --> 65 - 70%) haben einen deutlich höheren Anteil an Winterradlern als Zählstationen, die vornehmlich den Freizeitverkehr zählen (Poller Wiesen, Niederländer Ufer --> ca. 50%)
- Venloer Straße und Neumarkt zeigen einen Knick/Rückgang im Juli/August. Dies dürfte auf die Sommerferien und die geringere Anzahl an radfahrenden SchülerInnen zurückzuführen sein. Am stärksten ist dieser Rückgang übrigens für die Zählstation Stadtwald zu beobachten.

In [37]:
bike = load_bike_data(start, end, specific_days="all", remove_holidays=True)
(bike.groupby(bike.index.month).mean() / bike.mean()).iplot()

In [38]:
# Finale Grafik!
map_month={1:'Jan', 
           2:'Feb', 
           3:'Mrz', 
           4:'Apr', 
           5:'Mai', 
           6:'Jun', 
           7:'Jul',
           8: "Aug", 
           9: "Sept",
           10: "Okt",
           11: "Nov",
           12: "Dez"}
df = (bike.groupby(bike.index.month).mean() / bike.mean()).loc[:, ["Venloer Straße", "Neumarkt", "Poller Wiesen", "Niederländer Ufer"]]
df.index = df.index.map(map_month)
fig = px.line(df, 
              template=style,
              labels={"value": "Normierte monatliche Verkehrsstärke",  
                      "index": "Monat", 
                      "variable": "Zählstation"})
# Update title
fig.update_layout(
    title={'text': "Wie viele RadfahrerInnen werden pro Monat und Zählstation gezählt?",
           'y':0.99,'x':0.5, 
           'xanchor': 'center', 'yanchor': 'top'})
# Update legend
fig.update_layout(
    font_family="Rockwell",
    legend=dict(title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center")
)
fig.show()

In [39]:
specific_days = "weekday"
bike = load_bike_data(start=data_start, end=end, specific_days=specific_days, remove_holidays=True)
bike_grouped_by_month = bike.groupby(bike.index.month).mean()
winter = bike_grouped_by_month.loc[[1, 2, 3, 10, 11, 12], :].sum()
summer = bike_grouped_by_month.loc[4:9].sum()
winter_summer = (winter / summer).sort_values().multiply(100).round(1)
winter_summer.name = "Winter / Sommer"

#### Der Vergleich mit "gutes / schlechtes Wetter" ergibt ein sehr ähnliches Ergebnis
#### zum Quotienten Winter  / Sommer. Könnte man plotten, ist aber erstmal hier per Default nicht mit dabei.
bike_weather = load_data(
    start, end, specific_days=specific_days, remove_holidays=True)
good_weather = bike_weather[(bike_weather.tavg > 10) & (bike_weather.prcp < 5)]
good_weather_yearly = good_weather.groupby(
    by=[good_weather.index.year]).mean()
bad_weather = bike_weather[(bike_weather.tavg <= 10) | (bike_weather.prcp >= 5)]
bad_weather_yearly = bad_weather.groupby(
     by=[bad_weather.index.year]).mean()

bad_good_weather = (bad_weather.mean(axis=0) / 
                    good_weather.mean(axis=0)).drop(
    ["tavg", "wspd", "prcp"]).sort_values().multiply(100).round(1)
bad_good_weather.name = "Schlechtes Wetter / Gutes Wetter"
_ = pd.concat([winter_summer, bad_good_weather], axis=1)

df = winter_summer
fig = px.bar(df, 
              template=style,
              labels={"value": "Verhältnis RadfahrerInnen Winter zu Sommer [%]",  "index": "Zählstation"},
            )
fig.update_layout(barmode='group')
# Update title
fig.update_layout(
    showlegend=False,
    title={'text': "Wie viele RadfahrerInnen fahren im Winter im Vergleich zum Sommer?",'y':0.99,'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

## Einfluss des Jahres
- Der Radverkehr nimmt schneller zu als die Bevölkerung in Köln. Unter der Annahme, dass die Kölner nicht mobiler werden, heißt das, dass der Model Split sich zugunsten des Radverkehrs verändert.
- Das Wetter (insbesondere die Temperatur) hat (wie wir noch genauer analysieren weren) einen starken Einfluss auf das Radverkehrsaufkommen. Beispiele, bei denen das Wetter eine Rolle gespielt haben dürfte:
    - Die Stagnation der Bonner Straße zwischen 2014 und 2017 dürfte u.a. auf die im Vergleich zu 2014 niedrigeren Temperaturen in 2015 - 2017 zurückzuführen sein.
    - Der starke Anstieg von 2017 zu 2018 der Poller Wiesen dürfte u.a. durch das sehr gute Wetter in 2018 begründet sein.
- Analyse der einzelnen Zählstationen:
    - Bonner Straße: Kontinuierliche Steigerung. Wo in 2013 noch 85 RadfahrerInnen fuhren, fahren in 2020 schon 136. Eine Steigerung von 60% in 7 Jahren.
    - Poller Wiesen: Steigerung von 2016 bis 2020 um 73%. Exorbitante Zunahme von 2019 auf 2020. Grund dürfte die Corono-bedingte starke Zunahme des Freizeitverkehrs sein.
    - Neumarkt: Seit 2017 quasi Stagnation bzw. sogar leichter Rückgang.
- 2019 -- 2020: #Corona - Starke Zunahme des Freizeitverkehrs. Hingegen keine signifikante Zunahme (in absoluten Zahlen) des Berufsverkehrs.

In [40]:
start_h = start
end_h = datetime(2020, 12, 31)
specific_days_h = "all"
remove_holidays_h = False
bike = load_bike_data(start=start_h, 
                      end=end_h, 
                      specific_days=specific_days_h, 
                      remove_holidays=remove_holidays_h).loc[:, ["Venloer Straße", 
                                                                 "Neumarkt", 
                                                                 "Poller Wiesen", 
                                                                 "Niederländer Ufer", 
                                                                 "Bonner Straße",]]
bike_grouped_by_year = bike.groupby(bike.index.year).sum()
# Extrapoliere November und Dezember 2020 durch Werte aus dem Jahr 2019
bike_grouped_by_year.loc[2020] += bike.loc[datetime(2019, 11, 1): datetime(2019, 12, 31)].sum()
bike_grouped_by_year_normed = (bike_grouped_by_year / bike_grouped_by_year.loc[2016]).multiply(100).round(0)
bike_grouped_by_year_normed.loc[:, ["Venloer Straße", 
                                    "Neumarkt", 
                                    "Poller Wiesen", 
                                    "Niederländer Ufer"]][bike_grouped_by_year_normed.index < 2016] = np.nan

weather = load_weather_data(start=start_h,
                            end=end_h,
                            specific_days=specific_days_h,
                            remove_holidays=remove_holidays_h)
new_index = pd.date_range(start=start_h, end=datetime(2020, 12, 31), freq="D")
weather = weather.reindex(new_index)
weather.loc[datetime(2020, 11, 1): datetime(2020, 12, 31), "tavg"] = weather.loc[datetime(2019, 11, 1): datetime(2019, 12, 31), "tavg"].values
weather.loc[datetime(2020, 11, 1): datetime(2020, 12, 31), "prcp"] = weather.loc[datetime(2019, 11, 1): datetime(2019, 12, 31), "prcp"].values
tavg_grouped_by_year = weather.groupby(weather.index.year).mean().loc[:, ["tavg"]]
prcp_grouped_by_year = weather.groupby(weather.index.year).sum().loc[:, ["prcp"]]

bike_grouped_by_year_normed.loc[[2013, 2014, 2015], 
                                ["Venloer Straße", 
                                 "Neumarkt", 
                                 "Poller Wiesen", 
                                 "Niederländer Ufer"]] = np.nan

einwohner_koeln_normed = (einwohner_koeln / einwohner_koeln.loc[datetime(2016, 1, 1), :]).groupby(
    einwohner_koeln.index.year).mean().multiply(100).round(1)

fig = make_subplots(rows=2, cols=1,
                    specs=[[{"secondary_y": True}],
                           [{"secondary_y": True}]],
                   row_heights=[0.3, 0.7])
fig.add_trace(
    go.Bar(x=prcp_grouped_by_year.index, 
           y=prcp_grouped_by_year.prcp.values, 
           name="Jährlicher Niederschlag [mm]",
           legendgroup="Wetter"),
        row=1, 
        col=1, 
        secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=tavg_grouped_by_year.index, 
               y=tavg_grouped_by_year.tavg.values, 
               name="Jahresdurchschnittstemperatur [°C]",
               legendgroup="Wetter"),
            row=1, 
            col=1, 
            secondary_y=True)

textposition = "inside"
fig.add_trace(go.Bar(x=bike_grouped_by_year_normed.index, 
                     y=bike_grouped_by_year_normed["Neumarkt"].values, 
                     name="Neumarkt",
                     legendgroup="Zählstellen",
                     text=bike_grouped_by_year_normed["Neumarkt"].values,
                     textposition=textposition),
                    row=2, 
                    col=1, 
                    secondary_y=False)
fig.add_trace(go.Bar(x=bike_grouped_by_year_normed.index, 
                     y=bike_grouped_by_year_normed["Bonner Straße"].values, 
                     name="Bonner Straße",
                     legendgroup="Zählstellen",
                     text=bike_grouped_by_year_normed["Bonner Straße"].values,
                     textposition=textposition),
             row=2, col=1, secondary_y=False)
fig.add_trace(go.Bar(x=bike_grouped_by_year_normed.index, 
                     y=bike_grouped_by_year_normed["Poller Wiesen"].values, 
                     name="Poller Wiesen",
                     legendgroup="Zählstellen",
                     text=bike_grouped_by_year_normed["Poller Wiesen"].values,
                     textposition=textposition),
             row=2, col=1, secondary_y=False)

fig.add_trace(go.Scatter(x=einwohner_koeln_normed.index, 
                         y=einwohner_koeln_normed["einwohner"].values, 
                         name="Einwohnerentwicklung Köln",
                         legendgroup="Einwohner"),
              row=2, 
              col=1, 
              secondary_y=False,
              )

# Update yaxis properties
fig.update_yaxes(title_text="Niederschlag [mm]", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Temperatur [°C]", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="Normierte Verkehrsstärke", row=2, col=1)

# Update legend
fig.update_layout(
     font_family="Rockwell",
     legend=dict(title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center")
 )

fig.update_layout(
    autosize=False,
    width=900,
    height=650)

fig.show()

## Einfluss des Wetters
- "Berufsverkehr"-Zählpunkte
    - Je höher die Temperaturen, je niedriger der Niederschlag, desto mehr Menschen fahren mit dem Fahrrad.
    - Die Temperatur scheint einen etwas stärkeren Einfluss als der Niederschlag zu haben.
    - Eine Regressionsanalyse ergibt:
        - Steigt die Temperatur um 1°C, erhöht sich die Radverkehrsstärke um ca. 480 RadfahrerInnen/Tag
        - Selbst bei 0°C fahren immer noch ca. 10000 Personen pro Tag
    - Ab 20°c fahren jeden Tag zusammen über 18000 RadfahrerInnen pro Tag an den Stationen Neumarkt, Venloer, Straße, Hohenzollernbrücke und Bonner Straße.
- Wie viele Menschen fahren bei schlechtem Wetter (= T <= 10°C oder Niederschlag >= 10mm/Tag) im Vergleich zu gutem Wetter (= T > 10°C und Niederschlag < 10mm/Tag)?
    - Das Ergebnis entspricht ziemlich genau den Ergebnissen bei Vergleich der Jahreszeiten (Okt - Mrz / Apr - Sep). Bspw. fahren auf der Bonner Straße bei schlechtem Wetter noch 69% der Personen, die auch bei gutem Wetter Fahrrad fahren.


In [49]:
bike_weather = load_data(start=work_stations_data_start, end=end, specific_days="weekday", remove_holidays=True)
bike_weather = bike_weather[(bike_weather.prcp <= 20) & (bike_weather.tavg <= 30)]
bike = bike_weather[work_stations]
weather = bike_weather[["prcp", "tavg"]]

weather["prcp"] = weather["prcp"] - weather["prcp"].mod(1)
weather["tavg"] = weather["tavg"] - weather["tavg"].mod(1)
bike["sum"] = bike.sum(axis=1)
fig = go.Figure(data =
      go.Contour(x = weather["tavg"].values, 
                 y = weather["prcp"].values, 
                 z = bike["sum"].values,
                connectgaps=True,
                contours={"start": 10000, "end": 18000, "size": 4000})) 
fig.update_yaxes(title_text="Niederschlag [mm]")
fig.update_xaxes(title_text="Temperatur [°C]")
fig.show()

In [42]:
bike_weather = load_data(start=work_stations_data_start, end=end, specific_days="weekday", remove_holidays=True)
bike = bike_weather[work_stations]
bike["sum"] = bike.sum(axis=1)
fig = px.scatter(y=bike["sum"].values, x=bike_weather["tavg"].values)
# fig.show()
df = pd.DataFrame(index=bike_weather["tavg"].values, data=bike["sum"].values)
model = sm.OLS(bike["sum"].values, sm.add_constant(bike_weather["tavg"].values)).fit()
model.summary()    

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     990.4
Date:                Wed, 25 Nov 2020   Prob (F-statistic):          3.77e-147
Time:                        15:23:30   Log-Likelihood:                -8559.8
No. Observations:                 901   AIC:                         1.712e+04
Df Residuals:                     899   BIC:                         1.713e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.068e+04    210.937     50.609      0.000    1.03e+04    1.11e+04
x1           477.5150     15.174     31.470      0.000     447.735     507.295
==============================================================================
Omnibus:                       50.293   Durbin-Watson:                   0.912
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.484
Skew:                          -0.556   Prob(JB):                     1.21e-13
Kurtosis:                       3.592   Cond. No.                         27.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
bike_weather = load_data(
    start, end, specific_days="weekday", remove_holidays=True)
good_weather = bike_weather[(bike_weather.tavg > 10) & (bike_weather.prcp < 5)]
good_weather_yearly = good_weather.groupby(
    by=[good_weather.index.year]).mean()
bad_weather = bike_weather[(bike_weather.tavg <= 10) | (bike_weather.prcp >= 5)]
bad_weather_yearly = bad_weather.groupby(
     by=[bad_weather.index.year]).mean()
# Pro Station
# (bad_weather_yearly / good_weather_yearly).multiply(100).round(1)
(bad_weather.mean(axis=0) / good_weather.mean(axis=0)).sort_values().multiply(100).round(1)

tavg                   39.1
Poller Wiesen          45.9
Niederländer Ufer      46.4
Stadtwald              53.9
Deutzer Brücke         59.8
Vorgebirgspark         60.8
Neumarkt               65.2
Hohenzollernbrücke     65.8
Venloer Straße         67.2
Bonner Straße          69.3
wspd                  114.4
prcp                  646.1
dtype: float64

## Ausgewählte Tage
### Ergebnisse:
- Spitzenwerte:
    - Poller Wiesen (05.05.2016): Christi Himmelfahrt (Vatertag)
    - Deutzer Brücke + Neumarkt (10.04.2018): Verdi Warnstreik auf dem Heumarkt
    - Niederländer Ufer (14.06.2015): Birlikte2: Festival gegen Rassismus in Mülheim
- Niedrigste Werte:
    - Venloer Straße, Stadtwald, Neumarkt: 1. Weihnachtsfeiertag


In [44]:
# Max!
bike = load_bike_data(start, end, specific_days="all", remove_holidays=False)
max_idx = bike.idxmax().sort_values()
weather = load_weather_data(start, end, specific_days="all", remove_holidays=False)
reasons = {"Niederländer Ufer": "Birlikte2: Festival gegen Rassismus in Mülheim",
          "Poller Wiesen": "Christi Himmelfahrt (Vatertag)",
          "Deutzer Brücke": "Verdi Warnstreik auf dem Heumarkt",
          "Neumarkt": "Verdi Warnstreik auf dem Heumarkt",
          "Hohenzollernbrücke": "?",
          "Venloer Straße": "?",
          "Stadtwald": "?",
          "Vorgebirgspark": "?",
          "Bonner Straße": "?"}
for station, date in max_idx.items():
    print(f"Temperatur={weather.loc[date, 'tavg'].round(1)}," 
          f"Niederschlag={weather.loc[date, 'prcp'].round(1)}" 
          f"--> {station}, {date.strftime('%d.%m.%Y')}, " 
          f"Feiertag={date in get_holidays_and_brueckentage(start, end)}, "
          f"Grund: {reasons[station]}")

Temperatur=18.3,Niederschlag=0.0--> Niederländer Ufer, 14.06.2015, Feiertag=False, Grund: Birlikte2: Festival gegen Rassismus in Mülheim
Temperatur=12.9,Niederschlag=0.0--> Poller Wiesen, 05.05.2016, Feiertag=True, Grund: Christi Himmelfahrt (Vatertag)
Temperatur=14.6,Niederschlag=10.8--> Deutzer Brücke, 10.04.2018, Feiertag=False, Grund: Verdi Warnstreik auf dem Heumarkt
Temperatur=14.6,Niederschlag=10.8--> Neumarkt, 10.04.2018, Feiertag=False, Grund: Verdi Warnstreik auf dem Heumarkt
Temperatur=17.5,Niederschlag=0.0--> Hohenzollernbrücke, 18.04.2018, Feiertag=False, Grund: ?
Temperatur=21.1,Niederschlag=0.0--> Venloer Straße, 19.04.2018, Feiertag=False, Grund: ?
Temperatur=26.7,Niederschlag=0.0--> Stadtwald, 26.06.2019, Feiertag=False, Grund: ?
Temperatur=26.7,Niederschlag=0.0--> Vorgebirgspark, 26.06.2019, Feiertag=False, Grund: ?
Temperatur=21.0,Niederschlag=0.0--> Bonner Straße, 28.06.2019, Feiertag=False, Grund: ?


In [45]:
# Min!
bike = load_bike_data(start, end, specific_days="all", remove_holidays=False)
min_idx = bike.idxmin().sort_values()
weather = load_weather_data(start, end, specific_days="all", remove_holidays=False)
reasons = {"Niederländer Ufer": "?",
          "Poller Wiesen": "?",
          "Deutzer Brücke": "Samstag + Schlechtes Wetter",
          "Neumarkt": "Weihnachten",
          "Hohenzollernbrücke": "Schnee",
          "Venloer Straße": "Weihnachten",
          "Stadtwald": "Weihnachten",
          "Vorgebirgspark": "Weihnachten",
          "Bonner Straße": "Schlechtes Wetter + Rund um Köln?"}
for station, date in min_idx.items():
    print(f"Temperatur={weather.loc[date, 'tavg'].round(1)},  " 
          f"Niederschlag={weather.loc[date, 'prcp'].round(1)}" 
          f"--> {station}, {date.strftime('%d.%m.%Y')}, " 
          f"Feiertag={date in get_holidays_and_brueckentage(start, end)}, "
          f"Grund: {reasons[station]}")

Temperatur=9.7,  Niederschlag=4.3--> Venloer Straße, 25.12.2015, Feiertag=True, Grund: Weihnachten
Temperatur=9.7,  Niederschlag=4.3--> Stadtwald, 25.12.2015, Feiertag=True, Grund: Weihnachten
Temperatur=9.7,  Niederschlag=4.3--> Vorgebirgspark, 25.12.2015, Feiertag=True, Grund: Weihnachten
Temperatur=8.7,  Niederschlag=6.5--> Niederländer Ufer, 08.02.2016, Feiertag=False, Grund: ?
Temperatur=15.6,  Niederschlag=26.5--> Deutzer Brücke, 25.06.2016, Feiertag=False, Grund: Samstag + Schlechtes Wetter
Temperatur=2.3,  Niederschlag=2.5--> Poller Wiesen, 10.12.2017, Feiertag=False, Grund: ?
Temperatur=2.3,  Niederschlag=2.5--> Hohenzollernbrücke, 10.12.2017, Feiertag=False, Grund: Schnee
Temperatur=20.9,  Niederschlag=26.2--> Bonner Straße, 10.06.2018, Feiertag=False, Grund: Schlechtes Wetter + Rund um Köln?
Temperatur=2.4,  Niederschlag=0.0--> Neumarkt, 25.12.2018, Feiertag=True, Grund: Weihnachten
